#**ObjectTracking(YOLO-NAS_&_DeepSORT)**

#**Install All the Required Packages**

In [ ]:
!pip install -q super-gradients==3.2.1
!pip install filterpy==1.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 38.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to bui

#**Import All the Required Libraries**

In [ ]:
import cv2
import torch
from super_gradients.training import models
import numpy as np
import math
from numpy import random
from IPython.display import HTML
from base64 import b64encode
import os

[2024-01-31 18:25:26] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


[2024-01-31 18:25:27] WARNING - __init__.py - Failed to import pytorch_quantization
[2024-01-31 18:25:27] INFO - utils.py - NumExpr defaulting to 2 threads.
/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2024-01-31 18:25:40] WARNING - calibrator.py - Failed to import pytorch_quantization
[2024-01-31 18:25:40] WARNING - export.py - Failed to import pytorch_quantization
[2024-01-31 18:25:40] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2024-01-31 18:25:40] WARNING - env_sanity_check.py - Failed to verify installed packages: boto3 required but not found
[2024-01-31 18:25:40] WARNING - env_sanity_check.py - Failed to verify installed packages: deprecated required but not found
[2024-01-31 18:25:40] WARNING - env_sanity_check.py - Failed to verify installed packages: coverage required but not found
[2024-01-31 18:25:40] WARNING

#**Download the Demo Videos**

In [ ]:
!gdown "https://drive.google.com/uc?export=download&id=1Jhb9Kw2HfpOV8NwvG4I3rmJr3vzlGfXT"
!gdown "https://static.vecteezy.com/system/resources/previews/002/018/389/mp4/highway-traffic-merging-free-video.mp4"
!gdown "https://drive.google.com/uc?export=download&id=1IP0bG53LNrczN1dLuf7NmqXl0IuK1EnR"

Downloading...
From: https://drive.google.com/uc?export=download&id=1Jhb9Kw2HfpOV8NwvG4I3rmJr3vzlGfXT
To: /content/OTtest.mp4
100% 1.79M/1.79M [00:00<00:00, 112MB/s]
Downloading...
From: https://static.vecteezy.com/system/resources/previews/002/018/389/mp4/highway-traffic-merging-free-video.mp4
To: /content/highway-traffic-merging-free-video.mp4
4.89kB [00:00, 52.2MB/s]


#**Load the Input Video and calculate the Frame Width and Frame Height**

In [ ]:
def get_video_info(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        raise ValueError("Error: Could not open the video file.")

    # Get the frame width and frame height
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    return cap, frame_width, frame_height

In [ ]:
video_path = '/content/test4.mp4'
cap, frame_width, frame_height = get_video_info(video_path)
print(cap, frame_width, frame_height)


#**Download the Pre-Trained YOLO-NAS Model Weights**

In [ ]:
def load_model(model_name):
    # Check if CUDA (GPU) is available and set the device accordingly
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

    # Load the specified model
    model = models.get(model_name, pretrained_weights="coco").to(device)
    return model


In [ ]:
model_name = 'yolo_nas_s'
model = load_model(model_name)

Downloading...
From: https://drive.google.com/uc?export=download&id=1IP0bG53LNrczN1dLuf7NmqXl0IuK1EnR
To: /content/test4.mp4
100% 635k/635k [00:00<00:00, 126MB/s]
< cv2.VideoCapture 0x7a84f4021690> 854 480


[2024-01-31 18:25:45] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
Downloading: "https://sghub.deci.ai/models/yolo_nas_s_coco.pth" to /root/.cache/torch/hub/checkpoints/yolo_nas_s_coco.pth
100%|██████████| 73.1M/73.1M [00:00<00:00, 175MB/s]
[2024-01-31 18:25:45] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_s


#**1. Object Detection with YOLO-NAS**

# Classes in the COCO Dataset

In [ ]:
def cococlassNames():
  class_names = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella","handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat","baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup","fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli","carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed","diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone","microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors","teddy bear", "hair drier", "toothbrush" ]
  return class_names


In [ ]:
classNames = cococlassNames()

#Prediction

In [ ]:
out=cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 20, (frame_width, frame_height))
count=0
while True:
    ret, frame=cap.read()
    count+=1
    if ret:
        result=list(model.predict(frame, conf=0.5))[0]
        bbox_xyxys=result.prediction.bboxes_xyxy.tolist()
        confidences=result.prediction.confidence
        labels=result.prediction.labels.tolist()

        for (bbox_xyxy, confidence, cls) in zip(bbox_xyxys, confidences, labels):
            bbox=np.array(bbox_xyxy)
            x1, y1, x2, y2 = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
            classname=int(cls)
            class_name=classNames[classname]
            conf=math.ceil((confidence*100))/100
            label=f'{class_name}{conf}'
            print("Frame", count, "", x1, y1,x2, y2, label)

            #Create Bounding Boxes around the Detected Objects
            cv2.rectangle(frame, (x1, y1), (x2, y2), color=(0, 0, 255, 0.6),thickness=2, lineType=cv2.LINE_AA)

            #Create a rectangle above the detected object and add label and confidence score
            t_size=cv2.getTextSize(str(label), 0, fontScale=1/2, thickness=1)[0]
            c2=x1+t_size[0], y1-t_size[1]-3
            cv2.rectangle(frame, (x1, y1), c2, color=(0, 0, 255, 0.6), thickness=-1, lineType=cv2.LINE_AA)
            cv2.putText(frame, str(label), (x1, y1-2), 0, 1/2, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)

        out.write(frame)
    else:
        break
out.release()
cap.release()

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
[2024-01-31 18:25:45] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-31 18:25:47] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Frame 1  384 354 474 414 car0.9
Frame 1  0 411 83 468 car0.8
Frame 1  614 330 682 371 car0.8
Frame 1  182 320 351 437 truck0.79
Frame 1  341 359 393 404 car0.78
Frame 1  396 426 484 478 car0.71
Frame 1  455 346 534 402 car0.71
Frame 1  643 252 780 350 truck0.67
Frame 1  515 331 593 383 car0.63
Frame 1  52 391 136 438 car0.62
Frame 1  125 400 216 453 car0.61


[2024-01-31 18:25:49] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Frame 2  387 354 477 414 car0.9
Frame 2  2 409 98 467 car0.88
Frame 2  63 389 148 437 car0.85
Frame 2  344 360 396 403 car0.79
Frame 2  136 398 217 451 car0.79
Frame 2  480 342 544 398 car0.7
Frame 2  651 324 708 363 car0.68
Frame 2  542 327 626 378 car0.61
Frame 2  180 309 360 437 truck0.57
Frame 2  70 319 143 355 car0.57
Frame 2  756 183 829 240 truck0.56
Frame 2  412 272 476 314 car0.51


[2024-01-31 18:25:51] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


...

Frame 148  151 376 212 423 car0.81
Frame 148  227 383 309 425 car0.79
Frame 148  0 402 59 456 car0.71
Frame 148  285 356 321 394 car0.56
Frame 148  57 386 159 449 car0.53
Frame 148  481 60 532 101 bus0.51
Frame 148  691 313 755 352 car0.51


[2024-01-31 18:28:32] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Frame 149  171 367 269 419 car0.83
Frame 149  256 378 339 420 car0.82
Frame 149  0 401 93 457 car0.64
Frame 149  95 385 186 442 car0.6
Frame 149  312 328 427 408 truck0.57


[2024-01-31 18:28:33] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`


Frame 150  31 397 123 454 car0.88
Frame 150  286 374 358 414 car0.81
Frame 150  187 370 262 416 car0.8
Frame 150  631 316 696 358 car0.8
Frame 150  121 384 209 454 car0.79
Frame 150  714 404 796 453 car0.61
Frame 150  236 365 291 406 car0.61
Frame 150  555 308 625 356 car0.58
Frame 150  175 198 226 231 car0.54
Frame 150  462 337 531 381 car0.53
Frame 150  344 325 447 403 truck0.52
Frame 150  442 261 501 299 car0.52


#Display the Output Demo Video

In [ ]:
!rm '/content/result_compressed.mp4'

In [ ]:
def display_compressed_video(input_path):
    # Compressed video path
    compressed_path = "/content/result_compressed.mp4"

    os.system(f"ffmpeg -i {input_path} -vcodec libx264 {compressed_path}")

    # Show video
    mp4 = open(compressed_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display_html = """
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url

    return HTML(display_html)

In [ ]:
input_video_path = '/content/output.avi'
display_compressed_video(input_video_path)

Frame 151  680 301 738 352 car0.81
Frame 151  204 369 275 415 car0.8
Frame 151  56 393 144 451 car0.76
Frame 151  385 322 474 402 truck0.72
Frame 151  140 382 227 437 car0.72
Frame 151  304 366 382 413 car0.65
Frame 151  519 331 587 376 car0.62
Frame 151  602 226 654 262 car0.6
Frame 151  505 241 565 285 car0.59
Frame 151  642 216 685 249 car0.52
Frame 151  684 229 722 261 car0.51
rm: cannot remove '/content/result_compressed.mp4': No such file or directory


#**2.YOLO-NAS with DeepSORT Tracking**

#To implement the Object Tracking with DeepSORT, Download the DeepSORT  files

In [ ]:
!gdown "https://drive.google.com/uc?id=11ZSZcG-bcbueXZC3rN08CM0qqX3eiHxf&confirm=t"


Downloading...
From: https://drive.google.com/uc?id=11ZSZcG-bcbueXZC3rN08CM0qqX3eiHxf&confirm=t
To: /content/deep_sort_pytorch.zip
100% 43.1M/43.1M [00:00<00:00, 163MB/s]


In [ ]:
!unzip /content/deep_sort_pytorch.zip

Archive:  /content/deep_sort_pytorch.zip
  inflating: deep_sort_pytorch/.gitignore  
   creating: deep_sort_pytorch/configs/
  inflating: deep_sort_pytorch/configs/deep_sort.yaml  
   creating: deep_sort_pytorch/deep_sort/
  inflating: deep_sort_pytorch/deep_sort/__init__.py  
   creating: deep_sort_pytorch/deep_sort/__pycache__/
  inflating: deep_sort_pytorch/deep_sort/__pycache__/__init__.cpython-310.pyc  
  inflating: deep_sort_pytorch/deep_sort/__pycache__/__init__.cpython-37.pyc  
  inflating: deep_sort_pytorch/deep_sort/__pycache__/__init__.cpython-38.pyc  
  inflating: deep_sort_pytorch/deep_sort/__pycache__/deep_sort.cpython-310.pyc  
  inflating: deep_sort_pytorch/deep_sort/__pycache__/deep_sort.cpython-37.pyc  
  inflating: deep_sort_pytorch/deep_sort/__pycache__/deep_sort.cpython-38.pyc  
   creating: deep_sort_pytorch/deep_sort/deep/
 extracting: deep_sort_pytorch/deep_sort/deep/__init__.py  
   creating: deep_sort_pytorch/deep_sort/deep/__pycache__/
  inflating: deep_sort_

#Import all the DeepSORT Required Libraries

 get_config library  contains functions or classes for parsing configuration files and command-line arguments related to Deep SORT tracking

deep_sort_pytorch.deep_sort contains contains classes and functions for initializing the tracker, processing detections, associating detections with existing tracks, updating track states, and producing tracking results.


In [ ]:
from deep_sort_pytorch.utils.parser import get_config
from deep_sort_pytorch.deep_sort import DeepSort

#Load the Input Video and Calculate the Frame Width and Frame Height

In [ ]:
video_path = '/content/test4.mp4'
cap, frame_width, frame_height = get_video_info(video_path)


#Download the Pre-Trained YOLO-NAS Model Weights

In [ ]:
model_name = 'yolo_nas_l'
model = load_model(model_name)


  inflating: deep_sort_pytorch/deep_sort/deep/evaluate.py  
  inflating: deep_sort_pytorch/deep_sort/deep/feature_extractor.py  
  inflating: deep_sort_pytorch/deep_sort/deep/model.py  
  inflating: deep_sort_pytorch/deep_sort/deep/original_model.py  
  inflating: deep_sort_pytorch/deep_sort/deep/test.py  
  inflating: deep_sort_pytorch/deep_sort/deep/train.jpg  
  inflating: deep_sort_pytorch/deep_sort/deep/train.py  
  inflating: deep_sort_pytorch/deep_sort/deep_sort.py  
  inflating: deep_sort_pytorch/deep_sort/README.md  
   creating: deep_sort_pytorch/deep_sort/sort - Copy/
 extracting: deep_sort_pytorch/deep_sort/sort - Copy/__init__.py  
   creating: deep_sort_pytorch/deep_sort/sort - Copy/__pycache__/
  inflating: deep_sort_pytorch/deep_sort/sort - Copy/__pycache__/__init__.cpython-37.pyc  
  inflating: deep_sort_pytorch/deep_sort/sort - Copy/__pycache__/__init__.cpython-38.pyc  
  inflating: deep_sort_pytorch/deep_sort/sort - Copy/__pycache__/detection.cpython-37.pyc  
  infl

[2024-01-31 18:28:46] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
Downloading: "https://sghub.deci.ai/models/yolo_nas_l_coco.pth" to /root/.cache/torch/hub/checkpoints/yolo_nas_l_coco.pth
100%|██████████| 256M/256M [00:11<00:00, 23.8MB/s]
[2024-01-31 18:28:58] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


#Initialize the DeepSORT

Configuring and Initializing the instance of the Deep SORT tracker

In [ ]:
def initialize_deepsort():
    # Create the Deep SORT configuration object and load settings from the YAML file
    cfg_deep = get_config()
    cfg_deep.merge_from_file("/content/deep_sort_pytorch/configs/deep_sort.yaml")

    # Initialize the DeepSort tracker
    deepsort = DeepSort(cfg_deep.DEEPSORT.REID_CKPT,
                        max_dist=cfg_deep.DEEPSORT.MAX_DIST,
                        # min_confidence  parameter sets the minimum tracking confidence required for an object detection to be considered in the tracking process
                        min_confidence=cfg_deep.DEEPSORT.MIN_CONFIDENCE,
                        #nms_max_overlap specifies the maximum allowed overlap between bounding boxes during non-maximum suppression (NMS)
                        nms_max_overlap=cfg_deep.DEEPSORT.NMS_MAX_OVERLAP,
                        #max_iou_distance parameter defines the maximum intersection-over-union (IoU) distance between object detections
                        max_iou_distance=cfg_deep.DEEPSORT.MAX_IOU_DISTANCE,
                        # Max_age: If an object's tracking ID is lost (i.e., the object is no longer detected), this parameter determines how many frames the tracker should wait before assigning a new id
                        max_age=cfg_deep.DEEPSORT.MAX_AGE, n_init=cfg_deep.DEEPSORT.N_INIT,
                        #nn_budget: It sets the budget for the nearest-neighbor search.
                        nn_budget=cfg_deep.DEEPSORT.NN_BUDGET,
                        use_cuda=True
        )

    return deepsort

In [ ]:
deepsort = initialize_deepsort()

[2024-01-31 18:28:59] INFO - feature_extractor.py - Loading weights from deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7... Done!


#Creating Helping Functions

In [ ]:
names = cococlassNames()
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]
palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

### Function that adds fixed color depending on the class

In [ ]:
def compute_color_for_labels(label):
    if label == 0:  # person  #BGR
        color = (85, 45, 255)
    elif label == 2:  # Car
        color = (222, 82, 175)
    elif label == 3:  # Motobike
        color = (0, 204, 255)
    elif label == 5:  # Bus
        color = (0, 149, 255)
    else:
        color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)

In [ ]:
#Creating a helper function

def draw_boxes(img, bbox, identities=None, categories=None, names=None, offset=(0,0)):
    for i, box in enumerate(bbox):
        x1, y1,  x2, y2 = [int(i) for i in box]
        x1 += offset[0]
        x2 += offset[0]
        y1 += offset[0]
        y2 += offset[0]
        cat = int(categories[i]) if categories is not None else 0
        id = int(identities[i]) if identities is not None else 0

        #Create Bounding Boxes around the Detected Objects
        cv2.rectangle(img, (x1, y1), (x2, y2), color= compute_color_for_labels(cat),thickness=2, lineType=cv2.LINE_AA)
        label = str(id) + ":" + classNames[cat]
        (w,h), _ = cv2.getTextSize(str(label), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1/2, thickness=1)

        #Create a rectangle above the detected object and add label and confidence score
        t_size=cv2.getTextSize(str(label), cv2.FONT_HERSHEY_SIMPLEX, fontScale=1/2, thickness=1)[0]
        c2=x1+t_size[0], y1-t_size[1]-3
        cv2.rectangle(frame, (x1, y1), c2, color=compute_color_for_labels(cat), thickness=-1, lineType=cv2.LINE_AA)
        cv2.putText(frame, str(label), (x1, y1-2), 0, 1/2, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)

    return img

#Object Tracking with YOLO-NAS and DeepSORT

In [ ]:
classNames = cococlassNames()
output = cv2.VideoWriter('Output1.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width, frame_height))

while True:
    xywh_bboxs = []
    confs = []
    oids = []
    outputs = []
    ret, frame = cap.read()
    if ret:
        result = list(model.predict(frame, conf=0.5))[0]
        bbox_xyxys = result.prediction.bboxes_xyxy.tolist()
        confidences = result.prediction.confidence
        labels = result.prediction.labels.tolist()

        for (bbox_xyxy, confidence, cls) in zip(bbox_xyxys, confidences, labels):
            bbox = np.array(bbox_xyxy)
            x1, y1, x2, y2 = bbox[0], bbox[1], bbox[2], bbox[3]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            conf = math.ceil((confidence*100))/100
            cx, cy = int((x1+x2)/2), int((y1+y2)/2)
            bbox_width = abs(x1-x2)
            bbox_height = abs(y1-y2)
            xcycwh = [cx, cy, bbox_width, bbox_height]
            xywh_bboxs.append(xcycwh)
            confs.append(conf)
            oids.append(int(cls))

        xywhs = torch.tensor(xywh_bboxs)
        confss= torch.tensor(confs)
        outputs = deepsort.update(xywhs, confss, oids, frame)

        if len(outputs)>0:
            bbox_xyxy = outputs[:,:4]
            identities = outputs[:, -2]
            object_id = outputs[:, -1]
            draw_boxes(frame, bbox_xyxy, identities, object_id)
        output.write(frame)
    else:
        break

output.release()
cap.release()

[2024-01-31 18:28:59] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-31 18:29:03] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-31 18:29:05] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-31 18:29:08] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-31 18:29:11] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-31 18:29:14] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-01-31 18:29:18] INFO -

IndexError: too many indices for tensor of dimension 1

#Display the Output Video

In [ ]:
!rm '/content/result_compressed.mp4'

In [ ]:
input_video_path = '/content/Output1.avi'
display_compressed_video(input_video_path)
